# Библиотеки

In [1]:
import pandas as pd
import numpy as np
import datetime
import joblib

In [2]:
# Установка из драйва
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [3]:
from sklearn.cluster import KMeans ,AgglomerativeClustering

In [4]:
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import collections

In [5]:
!pip install PyDrive

In [6]:
!pip install yandex-geocoder

In [7]:
from decimal import Decimal
from yandex_geocoder import Client
#Key API (Вставить ключ API)
client = Client("9e1317de-547c-44c8-ab1b-b6b5a088ba91")

# Подтягиваем данные

In [8]:
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

urls= ['10kXYfu48klaNaMCtozYrGNJkZiHKLq5F','1S7VC63UML5K1TLYkpAjQdkYnosxEWEJc',
       '1alIPzv94_OaCSdqK2uhahjpSvRN98AyR','1paKd8b54f3PIQSHeNY7rsm3LpU9lEzkQ',
       '15lwZSD-OODXjh-UkkrBhh9R3EhfcRzLh','1sTcoYSqoU59wVV57WgTgEor018WqnNZx',
       '1xR_GqpKgFC5oBELWszTRjnsf28Usn0vb']

for c in urls:
    file_id = c
    downloaded = drive.CreateFile({'id':file_id})
    downloaded.FetchMetadata(fetch_all=True)
    downloaded.GetContentFile(downloaded.metadata['title'])

## Функции

Импорт данных

In [9]:
url = ['Каталог.xlsx', 'final_event.csv', 'org_hackaton.csv', 
       'final_book.csv', 'final_org.csv','model_knn_book',
       'model_knn_org']

def importData(url):
    dir = '/content/'
    # Каталог
    catalog = pd.read_excel(dir+url[0]).drop(columns=['p084a'])
    catalog['genre'] = catalog.p650a.str.replace('. ',';').str.split(';',expand=True)[0].copy()
    # Каталог
    event = pd.read_csv(dir+url[1])
    # Данные об организациях
    data3 = pd.read_csv(dir+url[2], sep=';')
    # Книги
    book_user = pd.read_csv(dir+url[3])
    # Кружки
    organiz = pd.read_csv(dir+url[4])
    # Импртируем МО
    model_book = joblib.load(dir+url[5])
    model_org = joblib.load(dir+url[6])
    return (catalog,event,data3,book_user,organiz,model_book,model_org)

Взаимодействие с пользователем Библиотек

In [10]:
def userInputB(hello, fronts, df):
    while hello:
        hello = input("У вас есть читательский номер? Введите 'Да' или 'Нет' ")
        if (hello == 'Да') | (hello == 'да'):
            id = int(input("Введите читательский номер: "))
            df1 = df[df['ИД читателя'] == id]
            if len(df1) == 0:
                return print('Неверный читательский номер. Отмена!')
            else:
                df1 = df1.drop(columns=['genre','age_range']).set_index(['ИД читателя'])  
                return (df1, df1['Возраст'].mean())
        elif (hello == 'Нет') | (hello == 'нет'):
            #print("Введите ваш возраст: ")
            age_1 = input("Введите ваш возраст: ")
            age_2 = age_range(age_1)
            ganre_1 = input("Какой жанр ищем? C большой буквы на русском: ")
            k = True
            while k:
                if ganre_1[:1] in fronts[0].values.tolist():
                    k = False
                else:
                    ganre_1 = input("Нет такого! Какой жанр ищем? C большой буквы на русском: ")
            period = input("Какое среднее время для чтения книги вам нужно в количестве дней: ")
            df1 = df[(df.classif == np.where(fronts[0] == ganre_1[:1])[0][0]) &
                     (df['Период'] < int(period)) &
                     (df.age_range_code == int(age_2))][:10].copy()
            df1['Возраст'] = int(age_1)
            df1 = df1.drop(columns=['genre','age_range']).set_index(['ИД читателя'])

            return (df1, df1['Возраст'].mean())

Взаимодействие с пользователем Кружков

In [11]:
def userInputO(hello, fronts, df):
    while hello:
        hello = input("Вам известен ваш id ученика? Введите 'Да' или 'Нет' ")
        if (hello == 'Да') | (hello == 'да'):
            id = int(input("Введите id ученика: "))
            df1 = df[df['id_ученика'] == id]
            if len(df1) == 0:
                return print('Неверный id ученика. Отмена!')
            else:
                df1 = df1.drop(columns=['Наименование_услуги','age_range']).set_index(['id_ученика']).astype(int)
                return (df1, df1['возраст'].mean())
        elif (hello == 'Нет') | (hello == 'нет'):
            #print("Введите ваш возраст: ")
            age_1 = input("Введите ваш возраст: ")
            age_2 = age_range(age_1)

            ganre_1 = input("Что вам нравится? C большой буквы на русском: ")
            k = True
            while k:
                if ganre_1[:1] in fronts[0].values.tolist():
                    k = False
                else:
                    ganre_1 = input("Нет такого! Какой жанр ищем? C большой буквы на русском: ")
            
            gender = input("Ваш пол: ")
            g = True
            while g:
                if (gender[:1] == 'M') | (gender[:1] == 'м'):
                    gender = 0
                    g = False
                elif (gender[:1] == 'Ж') | (gender[:1] == 'ж'):
                    gender = 1
                    g = False
                else:
                    gender = input("Это что ещё такое? Мужской или женский: ")
            
            df1 = df[(df.classif == np.where(fronts[0] == ganre_1[:1])[0][0]) &
                     (df['пол'] == gender) &
                     (df.age_range_code == int(age_2))][:10].copy()
            df1['возраст'] = int(age_1)
            df1 = df1.drop(columns=['Наименование_услуги','age_range']).set_index(['id_ученика']).astype(int)

            return (df1, df1['возраст'].mean())

### Прочее

Возраст

In [12]:
def age_range(val):
    val = val
    if val < 15:
        #val = 'до 14'
        val = 0
    elif val > 14 and val < 19:
        #val = 'от 14 до 18'
        val = 1
    elif val > 19 and val < 25:
        #val = 'от 18 до 24'
        val = 2
    elif val > 25 and val < 35:
        #val = 'от 25 до 34'
        val = 3
    elif val > 35 and val < 55:
        #val = 'от 35 до 55'
        val = 4
    else:
        #val = 'от 55'
        val = 5
    return val

Жанры

In [13]:
fronts = ['А', 'Б', 'В', 'Г', 'Д', 'Е', 'Ж', 'З', 'И', 'К', 'Л', 'М', 'Н',
          'О', 'П', 'Р', 'С', 'Т', 'У', 'Ф', 'Х', 'Ц', 'Ч', 'Ш', 'Э', 'Ю', 'Я']
fronts = pd.DataFrame(fronts).reset_index()

Предсказание похожих Книг

In [14]:
def freqInfo(df, search, col):
    result = []
    for c in search:
        result.append(df[df.index == df.index[c]][col].drop_duplicates().values.tolist())
        
    recom_list = []
    for i in range(len(result)):
        for j in range(len(result[i])):
            recom_list.append(result[i][j])
    counter=collections.Counter(recom_list)
    return (sorted(counter, key=counter.get, reverse=True)[:20])

In [15]:
def searchBook(df, train, bone, model_knn, col, data3, catalog, mode):
    search = []
    for i in range(len(df)):
        #Определяем дистанцию между авторами
        distances, indices = model_knn.kneighbors(df.iloc[i].values.reshape(1, -1), n_neighbors = 20)
        for j in range(0, len(indices.flatten())):
            search.append(indices.flatten()[j])
    
    counter=collections.Counter(search)
    search = sorted(counter, key=counter.get, reverse=True)[:20]
    
    result = pd.DataFrame([])

    if mode == 1:
        for c in freqInfo(train, search, col):
            result = result.append(train[train[col] == c])
        if col == 'ИД Каталожной записи':
            result = pd.merge(result[col],catalog,how='left',left_on=
                              col,right_on='doc_id').drop(columns=['doc_id','genre']).drop_duplicates()
        else:
            result = result[['Наименование_услуги', col,'years', 'weeks', 'months', 'days']]
            result = pd.merge(result, data3, how='left', on=col)

    elif mode == 2: 
        for c in freqInfo(train, search, 'age_range_code'):
            for cl in freqInfo(train, search, 'classif'):
                result = result.append(bone[(bone.age_range_code == c) & (bone.classif == cl)])
        if col == 'id_организации':
            result = result[['Наименование_услуги', col]].drop_duplicates()
            result = pd.merge(result, data3, how='left', on=col)
        else:
            result = pd.merge(result[col],catalog,how='left',left_on=col,right_on='doc_id').drop(columns=['doc_id','genre'])

    return (result)

Местоположение

In [16]:
def kmeanModel(position, event, age):
    result = event[(event['from'] < age) & (event['to'] > age)].copy()
    #Создает 125  кластеров, используя average кластеризацию. (Районов в Москве)
    agc = AgglomerativeClustering(n_clusters =125 , affinity='cosine', linkage='average')
    result['pickup cluster'] = agc.fit_predict(result[['longitude','latitude']])
    
    #Создает 125  кластеров, используя алгоритм кластеризации k-средних.
    kmeans = KMeans(125)
    clusters = kmeans.fit_predict(result[['longitude','latitude']])
    result['pickup cluster'] = kmeans.predict(result[['longitude','latitude']])

    longitude = float(client.coordinates(position + "Москва")[1])
    latitude = float(client.coordinates(position + "Москва")[0])

    result['distance'] = result.apply(haversine_distance, axis = 1)
    result = result.sort_values(by='distance', ascending=True)[:50]
    result = result.sort_values(by='Дата начала мероприятия', ascending=True)[:25]
    result = result.drop(columns=['id', 'Статус', 'Код категории мероприятия','from','to','latitude',	'longitude', 'pickup cluster', 'distance'])
    
    return result

In [17]:
def haversine_distance(row):
    lat_p, lon_p = row['longitude'], row['latitude']
    lat_d, lon_d = longitude,latitude
    radius =  6371 # km

    dlat = np.radians(lat_d - lat_p)
    dlon = np.radians(lon_d - lon_p)
    a = np.sin(dlat/2) * np.sin(dlat/2) + np.cos(np.radians(lat_p)) * np.cos(np.radians(lat_d)) * np.sin(dlon/2) * np.sin(dlon/2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    distance = radius * c

    return distance

# Рекомендательный алгоритм

Параметры пользователя (опрос)

Рекомендации

In [ ]:
catalog, event, data3, book_user, organiz, model_book, model_org = importData(url)

In [92]:
print("Для того, чтобы протестировать рекомендательный сервис, необходимо выбрать режим:")
print()
print("Режим по истории посещения библиотек, порекомендовать книги?")
print("Нажми 1")
print("Режим по истории посещения библиотек, порекомендовать ближайшие мероприятия и кружки?")
print("Нажми 2")
print("Режим по истории посещения кружков, порекомендовать кружки?")
print("Нажми 3")
print("Режим по истории посещения кружков, порекомендовать ближайшие мероприятия и подходящие по интересам книги?")
print("Нажми 4")

tps = input("Что ищем? ")
if int(tps) < 3:
    df, age = userInputB(True, fronts, book_user)
    col = 'ИД Каталожной записи'
    mode = int(tps)
    train = book_user
    bone = organiz
    model = model_book
    if mode == 1:
        col = 'ИД Каталожной записи'
    else:
        col = 'id_ученика'
else:
    df, age = userInputO(True, fronts, organiz)
    mode = int(tps) // 2
    train = book_user
    bone = organiz
    model = model_org
    if mode == 1:
        col = 'ИД Каталожной записи'
    else:
        col = 'id_ученика'

result = searchBook(df, train, bone, model, col, data3, catalog, mode)
result

Для того, чтобы протестировать рекомендательный сервис, необходимо выбрать режим:

Режим по истории посещения библиотек, порекомендовать книги?
Нажми 1
Режим по истории посещения библиотек, порекомендовать ближайшие мероприятия и кружки?
Нажми 2
Режим по истории посещения кружков, порекомендовать кружки?
Нажми 3
Режим по истории посещения кружков, порекомендовать ближайшие мероприятия и подходящие по интересам книги?
Нажми 4
Что ищем? 3
Вам известен ваш id ученика? Введите 'Да' или 'Нет' нет
Введите ваш возраст: 24


TypeError: ignored

In [ ]:
print('Заодно посмотрим ближайшие мероприятия по местоположению')
position = input("Какой адрес интересует? ")

result_event = kmeanModel(position, event, age)
result_event

# Написание бота

In [57]:
!pip install pytelegrambotapi

     |████████████████████████████████| 81kB 6.2MB/s 
  Created wheel for pytelegrambotapi: filename=pyTelegramBotAPI-3.7.3-cp36-none-any.whl size=55590 sha256=26dceb4d49a7bff3f29c50c88ac50ed079f76f22b0df3e3499a2fa7c8265df2e
  Stored in directory: /root/.cache/pip/wheels/77/19/9c/cf58e8c83abec055f875508722503d29cdb6a34f4c58c7537a
Successfully built pytelegrambotapi


In [81]:
import telebot
from telebot import types

bot = telebot.TeleBot(token='')

# Обычный режим
@bot.message_handler(content_types=["text"])
def any_msg(message):
    keyboard = types.InlineKeyboardMarkup()
    callback_button1 = types.InlineKeyboardButton(text="1", callback_data="test1")
    callback_button2 = types.InlineKeyboardButton(text="2", callback_data="test2")
    callback_button3 = types.InlineKeyboardButton(text="3", callback_data="test3")
    callback_button4 = types.InlineKeyboardButton(text="4", callback_data="test4")
    
    keyboard.add(callback_button1, callback_button2,callback_button3, callback_button4)
    bot.send_message(message.chat.id, "Я – сообщение из обычного режима", reply_markup=keyboard)


# В большинстве случаев целесообразно разбить этот хэндлер на несколько маленьких
@bot.callback_query_handler(func=lambda call: True)
def callback_inline(call):
    # Если сообщение из чата с ботом
    if call.message:
        if call.data == "test1":
            df, age = userInputB(True, fronts, book_user)
            col = 'ИД Каталожной записи'
            mode = int(tps)
            train = book_user
            bone = organiz
            model = model_book 
            result = searchBook(df, train, bone, model, col, data3, catalog, mode)
            text = event['Место проведения'].iloc[1] + ' ' +event['Направленность мероприятия'].iloc[1]
            bot.edit_message_text(chat_id=call.message.chat.id, message_id=call.message.message_id, text=text)
        elif call.data == "test2":
            text = event['Место проведения'].iloc[2] + ' ' +event['Направленность мероприятия'].iloc[2]
            bot.edit_message_text(chat_id=call.message.chat.id, message_id=call.message.message_id, text=text)

if __name__ == '__main__':
    bot.infinity_polling()